In [9]:
from keras.models import load_model
import numpy as np
import json

In [10]:
model = load_model("abalone_model.keras")

In [11]:
predict_input = np.array([[0.475,0.37,0.125,0.5095,0.2165,0.1125,0.165,9]])
predict_result = model.predict(predict_input)

json.dumps({"predict_result": predict_result.tolist()})

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 412ms/step


'{"predict_result": [[0.0]]}'

In [12]:
!pip install flask

In [14]:
import scipy.sparse
from sagemaker import image_uris
image_uris.retrieve(framework='tensorflow',region='us-east-1',version='1.12.0',image_scope='inference',instance_type='ml.c5.4xlarge')

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


'520713654638.dkr.ecr.us-east-1.amazonaws.com/sagemaker-tensorflow-serving:1.12.0-cpu'

In [15]:
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 520713654638.dkr.ecr.us-east-1.amazonaws.com

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [13]:
%%sh
# Specify an image name
image_name=tensorflow-inference
echo "image_name: ${image_name} ######################"

account=$(aws sts get-caller-identity --query Account --output text)
echo "account: ${account} ######################"

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
echo "region: ${region} ######################"

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${image_name}:2.12.0-cpu-py310-pluto-2"
echo "fullname: ${fullname} ######################"

aws ecr describe-repositories --repository-names "${image_name}"
if [ $? -ne 0 ]
then
aws ecr create-repository --repository-name "${image_name}"
fi

image_name: tensorflow-inference ######################
account: 891377019371 ######################
region: us-east-1 ######################
fullname: 891377019371.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference:2.12.0-cpu-py310-pluto-2 ######################



An error occurred (RepositoryNotFoundException) when calling the DescribeRepositories operation: The repository with name 'tensorflow-inference' does not exist in the registry with id '891377019371'


{
    "repository": {
        "repositoryArn": "arn:aws:ecr:us-east-1:891377019371:repository/tensorflow-inference",
        "registryId": "891377019371",
        "repositoryName": "tensorflow-inference",
        "repositoryUri": "891377019371.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference",
        "createdAt": 1730815819.268,
        "imageTagMutability": "MUTABLE",
        "imageScanningConfiguration": {
            "scanOnPush": false
        },
        "encryptionConfiguration": {
            "encryptionType": "AES256"
        }
    }
}


In [24]:
!docker build -t 891377019371.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference:2.12.0-cpu-py310-pluto-2 -f Dockerfile.inference .

[+] Building 0.0s (0/1)                                          docker:default
[+] Building 0.1s (5/5) FINISHED                                 docker:default
 => [internal] load build definition from Dockerfile.inference             0.0s
 => => transferring dockerfile: 137B                                       0.0s
 => [internal] load metadata for 520713654638.dkr.ecr.us-east-1.amazonaws  0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => CACHED [1/1] FROM 520713654638.dkr.ecr.us-east-1.amazonaws.com/sagema  0.0s
 => exporting to image                                                     0.0s
 => => exporting layers                                                    0.0s
 => => writing image sha256:6c4e927c704137ac6abf0876ca240f65dd16e9806ef1f  0.0s
 => => naming to 891377019371.dkr.ecr.us-east-1.amazonaws.com/tensorflow-  0.0s


In [25]:
!docker images

REPOSITORY                                                                  TAG                        IMAGE ID       CREATED       SIZE
520713654638.dkr.ecr.us-east-1.amazonaws.com/sagemaker-tensorflow-serving   1.12.0-cpu                 27aee10a9f6a   5 years ago   377MB
891377019371.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference           2.12.0-cpu-py310-pluto-2   6c4e927c7041   5 years ago   377MB


In [27]:
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 891377019371.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference:2.12.0-cpu-py310-pluto-2

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [30]:
!docker push 891377019371.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference:2.12.0-cpu-py310-pluto-2

The push refers to repository [891377019371.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference]

ab352ca1: Preparing 
1bba5efb: Preparing 
bcb4634f: Preparing 
ebf683ff: Preparing 
9781a77c: Preparing 
13a20bf2: Preparing 
cdf4e497: Preparing 
3f9454c0: Preparing 
6b5893dc: Pushed   126.2MB/122.2MBPushing   13.9MB/108.1MB2.12.0-cpu-py310-pluto-2: digest: sha256:2741292c10cfa163e6ba91c0e83100bb8debeac8a2213666b12f1f2560df7d12 size: 2203


In [ ]:
from sagemaker import get_execution_role
from sagemaker.estimator import Estimator
import os

role=get_execution_role()

hyperparameters={'epochs': 1}

estimator=Estimator(
    image_uri='891377019371.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:2.12.0-cpu-py310-housing',
    role=role,
    instance_count=1,
    instance_type='ml.p2.xlarge',
    hyperparameters=hyperparameters,
    output_path='s3://itsarghisdata/data/output'.format(os.getcwd())
)

print('##### ESTIMATOR FIT STARTED')
estimator.fit('s3://itsarghisdata/data/input/train.csv'.format(os.getcwd()))
print('##### ESTIMATOR FIT COMPLETED')